<a href="https://colab.research.google.com/github/Vaishnavi3041/Customer_feedback_analyser/blob/main/Nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask scikit-learn

from flask import Flask, render_template_string, request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from threading import Thread
from google.colab import output
import numpy as np


# -------------------------------
# TRAIN SENTIMENT MODEL
# -------------------------------

train_texts = [
    "I love this product, it's amazing",
    "Customer service was very helpful",
    "The delivery was quick and smooth",
    "Great quality, totally satisfied",
    "Excellent experience with this brand",
    "The product works perfectly",

    "The product arrived damaged",
    "Very poor customer service",
    "It took too long to deliver",
    "The item does not match the description",
    "Terrible quality, not worth the money",
    "I am very disappointed with the purchase",

    "The product is okay but nothing special",
    "Average quality for the price",
    "Delivery was fine, not too fast or slow",
    "Customer service was acceptable",
    "The product works but could be better"
]

train_labels = [
    "Positive","Positive","Positive","Positive","Positive","Positive",
    "Negative","Negative","Negative","Negative","Negative","Negative",
    "Neutral","Neutral","Neutral","Neutral","Neutral"
]

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)
model = LogisticRegression()
model.fit(X_train, train_labels)

emoji_map = {
    "Positive": "🌸😊✨",
    "Negative": "💔😣⚠️",
    "Neutral":  "😐🌙"
}

sentiment_counter = {"Positive": 0, "Negative": 0, "Neutral": 0}

# -------------------------------
# CHART GENERATION
# -------------------------------
def generate_chart():
    labels = ["Positive", "Negative", "Neutral"]
    values = [sentiment_counter["Positive"],
              sentiment_counter["Negative"],
              sentiment_counter["Neutral"]]

    fig, ax = plt.subplots(figsize=(5, 4))

    # Plot bars normally (no gradient)
    bars = ax.bar(labels, values)

    # Set colors and background
    ax.set_facecolor("#ffe6f2")
    fig.patch.set_facecolor("#ffe6f2")

    # Title
    ax.set_title("Sentiment Distribution", fontsize=14, color="#b35a7a")

    # Remove scientific notation on axis
    ax.get_yaxis().get_major_formatter().set_useOffset(False)

    # Slight pastel style
    for bar in bars:
        bar.set_edgecolor("#b35a7a")
        bar.set_linewidth(2)

    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format="png", dpi=120)
    buf.seek(0)
    img_b64 = base64.b64encode(buf.read()).decode("utf-8")
    plt.close()

    return img_b64




# -------------------------------
# FLASK APP WITH PASTEL UI
# -------------------------------

app = Flask(__name__)

html_template = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Pastel Sentiment Analyzer 🌸</title>

    <link href="https://fonts.googleapis.com/css2?family=Press+Start+2P&family=Quicksand:wght@400;600&display=swap" rel="stylesheet">

    <style>
        body {
            margin: 0;
            background: linear-gradient(180deg, #f7c9e3 0%, #c9d9ff 100%);
            font-family: 'Quicksand', sans-serif;
            height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
        }

        .cloud {
            position: absolute;
            width: 200px;
            opacity: 0.25;
            animation: float 16s infinite linear;
            filter: blur(1px);
        }
        @keyframes float {
            0% { transform: translateX(-20px); }
            50% { transform: translateX(20px); }
            100% { transform: translateX(-20px); }
        }

        .window {
            width: 430px;
            background: #ffeccd;
            border: 3px solid #d5a866;
            border-radius: 14px;
            box-shadow: 0px 6px 0px #b78d52;
        }

        .titlebar {
            background: #f5d19b;
            padding: 12px;
            font-family: 'Press Start 2P';
            font-size: 12px;
            color: #5a4224;
            border-bottom: 3px solid #d5a866;
            border-radius: 10px 10px 0 0;
            display: flex;
            justify-content: space-between;
        }

        .close-btn {
            width: 18px;
            height: 18px;
            background: #ff6b6b;
            border: 2px solid #c44141;
            border-radius: 5px;
        }

        .content {
            padding: 25px;
            text-align: center;
        }

        textarea {
            width: 85%;
            height: 90px;
            background: #fff3e3;
            border: 2px solid #d5a866;
            border-radius: 10px;
            padding: 10px;
            font-size: 15px;
        }

        button {
            margin-top: 12px;
            background: #ff9bbb;
            border: 2px solid #d46a85;
            border-radius: 10px;
            padding: 10px 25px;
            font-size: 16px;
            cursor: pointer;
            transition: 0.15s;
        }
        button:hover {
            background: #ff7aa3;
            transform: scale(1.05);
        }

        .result-box {
            margin-top: 18px;
            padding: 15px;
            background: #fff3e3;
            border: 2px solid #d5a866;
            border-radius: 12px;
        }

        /* LOADING OVERLAY */
        #loading {
            display: none;
            position: fixed;
            top: 0; left: 0;
            width: 100%; height: 100%;
            background: rgba(255,240,250,0.85);
            justify-content: center;
            align-items: center;
            z-index: 999;
        }

        .loading-box {
            width: 170px;
            height: 170px;
            background: #ffe1ef;
            border: 3px solid #e6a3c9;
            border-radius: 20px;
            box-shadow: 0px 6px 0px #d48ab2;
            display: flex;
            justify-content: center;
            align-items: center;
            animation: pop 1s infinite;
        }

        @keyframes pop {
            0% { transform: scale(1); }
            50% { transform: scale(1.07); }
            100% { transform: scale(1); }
        }

        .loading-text {
            font-family: 'Press Start 2P';
            font-size: 12px;
            color: #9b4d75;
            text-align: center;
        }
    </style>

    <script>
        function startLoading() {
            document.getElementById("loading").style.display = "flex";
        }
    </script>

</head>

<body>

    <!-- Floating Clouds (Base64 so they ALWAYS load) -->
<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOEAAADhCAMAAAAJbSJIAAAAkFBMVEUAAAD///////////////////////////////////////////////////////////////////////////////////////////////////////////////9d+jrYAAAALHRSTlMAAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYGRobHB0eHyAhIiMkJSYnKCstLS4vMDIxMzQ1NjwcrPwAAAFvSURBVHhe7d2JkoMgDABQJQpRkG3t+/9f7CEN0zjWE/S06KqUu7JLOy5I85mgOUEQhISEhISEhISEhISEhISEhISEhISEhISHhQ7p362eWhv14jArlTcidH833Mu6R0QlmrJxn7F4tvjo2WEhYvWARN1CMV6FZ3LqA77gksSmgZPUCzSkQ53KutG8zEFPv5LXoZxT3uR3KvgE+fS4g7dR+Y2lPw0zOeJd4R8Yml/QNjGeptXgHxieH3DexrHY6HWyQAAAABJRU5ErkJggg=="
     class="cloud" style="top:10%; left:5%;">

<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOEAAADhCAMAAAAJbSJIAAAAkFBMVEUAAAD///////////////////////////////////////////////////////////////////////////////////////////////////////////////9d+jrYAAAALHRSTlMAAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYGRobHB0eHyAhIiMkJSYnKCstLS4vMDIxMzQ1NjwcrPwAAAFvSURBVHhe7d2JkoMgDABQJQpRkG3t+/9f7CEN0zjWE/S06KqUu7JLOy5I85mgOUEQhISEhISEhISEhISEhISEhISEhISEhISHhQ7p362eWhv14jArlTcidH833Mu6R0QlmrJxn7F4tvjo2WEhYvWARN1CMV6FZ3LqA77gksSmgZPUCzSkQ53KutG8zEFPv5LXoZxT3uR3KvgE+fS4g7dR+Y2lPw0zOeJd4R8Yml/QNjGeptXgHxieH3DexrHY6HWyQAAAABJRU5ErkJggg=="
     class="cloud" style="bottom:10%; right:5%;">


    <div id="loading">
        <div class="loading-box">
            <div class="loading-text">Loading... ✨</div>
        </div>
    </div>

    <div class="window">
        <div class="titlebar">
            <span>Sentiment Window 💛</span>
            <div class="close-btn"></div>
        </div>

        <div class="content">
            <h3 style="color:#6b4e2e;">How are they feeling today? 🌸</h3>

            <form method="POST">
                <textarea name="text" placeholder="Type feedback...">{{ text }}</textarea><br>
                <button type="submit" onclick="startLoading()">Analyze 💫</button>
            </form>

            {% if prediction %}
            <div class="result-box">
                <p><strong>Text:</strong> {{ text }}</p>
                <p><strong>Sentiment:</strong> {{ prediction }}</p>
                <p style="font-size:30px;">{{ emoji }}</p>

                {% if chart %}
                <div style="margin-top:20px;">
                    <img src="data:image/png;base64,{{ chart }}" style="width:90%; border-radius:12px;">
                </div>
                {% endif %}
            </div>
            {% endif %}
        </div>
    </div>

</body>
</html>
"""

# -------------------------------
# FLASK ROUTE
# -------------------------------

@app.route("/", methods=["GET", "POST"])
def index():
    text = ""
    prediction = None
    emoji = ""
    chart = None

    if request.method == "POST":
        text = request.form["text"]
        X_test = vectorizer.transform([text])
        prediction = model.predict(X_test)[0]
        emoji = emoji_map[prediction]

        sentiment_counter[prediction] += 1
        chart = generate_chart()

    return render_template_string(html_template,
                                 text=text,
                                 prediction=prediction,
                                 emoji=emoji,
                                 chart=chart)


# -------------------------------
# RUN IN COLAB
# -------------------------------
def run_app():
    app.run(host="0.0.0.0", port=5000)

Thread(target=run_app).start()
output.serve_kernel_port_as_iframe(5000, height=600)

 * Serving Flask app '__main__'


<IPython.core.display.Javascript object>

 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
